# Entrenamiento de un modelo para el análisis del comportamiento de los clientes de Megaline

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Carga de liberías y datos](#data_review)
* [Segmentación de datos](#seg)
* [Entrenamiento de modelos](#train)
    * [Árbol de decisión](#tree)
    * [Bosque aleatorio](#forest)
    * [Regresión logística](#reg)
* [Calidad del modelo](#cal)
* [Conclusión general](#conclusion)

## Introducción <a id='intro'></a>

La compañía móvil Megaline quiere desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Se desarrollará un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75.

## Carga de librerias y datos <a id='data_review'></a>

In [1]:
# Cargar todas las librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


- La característica que necesitamos predecir es is_ultra, lo cual será el objetivo. 
- Las demás columnas serán las observaciones para predecir el objetivo.
- El objetivo es categórico ya que se recomendará el plan ultra (0) o megaline (1).

## Segmentación de datos <a id='seg'></a>

Como no contamos con un conjunto de prueba, se dividirán los datos fuente en proporción 3:1:1, un 75% para el conjunto de entrenamiento, 20% para el conjunto de validación y 20% para el conjunto de prueba.

In [5]:
df_train, df_valid_test = train_test_split(df, test_size = 0.4, random_state = 12233)

In [6]:
df_valid, df_test = train_test_split(df_valid_test, test_size = 0.5, random_state = 12233)

Determinamos las varibles de observacion y objetivo para cada conjunto de datos.

In [7]:
#variables para las características y para la característica objetivo 

#entrenamiento
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
#validacion
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
#prueba
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Entrenamiento de modelos <a id='train'></a>

Se probará tres modelos con algoritmos de clasificación, y se determinará que hiperparámetros es mejor para el modelo.

### Modelo árbol de decisión <a id='tree'></a>

Para este modelo se determinará a qué profundidad máxima funciona mejor el modelo por medio de la exactitud.

In [8]:
for depth in range(1, 6):
        model = DecisionTreeClassifier(random_state = 12233, max_depth=depth)
        model.fit(features_train, target_train)

        predictions_valid = model.predict(features_valid)

        print("max_depth =", depth, ": ", end='')
        print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7667185069984448
max_depth = 2 : 0.7900466562986003
max_depth = 3 : 0.80248833592535
max_depth = 4 : 0.7962674961119751
max_depth = 5 : 0.7978227060653188


La profundidad máxima que da mayor exactitud es 3.

### Modelo bosque aleatorio <a id='forest'></a>

Para determinar el número de árboles en este modelo se utilizará el hiperparámetro n_estimators.

In [13]:
best_score = 0
best_est = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12233, n_estimators=est)
    model.fit(features_train, target_train) 
    score = model.score(features_valid, target_valid) 
    if score > best_score:
        best_score = score
        best_est = est

print("Accuracy del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))


Accuracy del mejor modelo en el conjunto de validación (n_estimators = 10): 0.7993779160186625


### Modelo regresión logística <a id='reg'></a>

Para este modelo de regresión se utilizará una clasificación lineal para que funcione y se comprobarña la exactitud también en el conjunto de validación para comprobar si está sobreajustado el modelo.

In [14]:
model = LogisticRegression(random_state=12233,solver='liblinear')
model.fit(features_train, target_train) 
score_train = model.score(features_train, target_train)
score_valid = model.score(features_valid,target_valid) 

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.6950207468879668
Accuracy del modelo de regresión logística en el conjunto de validación: 0.6998444790046656


*De acuerdo al umbral de exactitud (0.75) el mejor algoritmo a utilizar es el árbol de decisión ajustado con el parámetro de profundidad másxima = 3.*

## Calidad del modelo <a id='cal'></a>

Para comprobar que el modelo funcione bien en el conjunto de prueba tambien, evaluamos su exactitud.

In [15]:
model = DecisionTreeClassifier(random_state = 12233, max_depth = 3) 
model.fit(features_train, target_train)

train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('Exactitud')
print('Training set:', accuracy_score(target_train, train_predictions))
print('Test set:', accuracy_score(target_test, test_predictions))

Exactitud
Training set: 0.7966804979253111
Test set: 0.7962674961119751


Funciona bien en el conjunto de prueba por lo tanto predice correctamente dentro del umbral de exactitud.

## Conclusión <a id='conclusion'></a>

En conclusión, el mejor modelo para el conjunto de datos fue el árbol de decisión con profundidad máxima en 3. 
- Los hiperparámetros afectaron la calidad de predidcción.
- Se obtuvo una exactitud similar en los conjuntos de entremiento, validación y prueba, por lo tanto el modelo no estaba sobreajustado y predice correctamente en campo.